# Test Analyser

Read in the results data (from `data/results.csv` and analyse it. We use the [TAM](https://cran.r-project.org/web/packages/TAM/) package as it's capable of dealing with tests that have a mix of dichotomous and polytomous items. The goal here is to analyse a simulated test to ensure that it approximates a real world test.

We assume that the 1PL model is used.

$$
Pr(X=1) = \frac{exp(\theta-b)}{1 + exp(\theta-b)}
$$

We'll want to use the partial credit model (PCM) to analyse the data. Items are classed by the number of categories ($k$) that they contain; dichotomous items have a value of $k = 2, K = \{ 0, 1 \}$. The polytomous items have a varying number of categories.

I've found [chapter 4 of Introduction to Rasch Analysis in R](https://bookdown.org/dkatz/Rasch_Biome/polytomous-items.html) to be useful in understanding how to analyse polytomous tests using TAM.

## Data Ingest

For this analysis we need the `results.csv` file from the `data` folder.

In [ ]:
library(tidyverse)
library(TAM)

dfRaw <- read_csv("data/results.csv")

dfTest <- dfRaw %>%
  dplyr::select(-systemname)

numItems <- ncol(dfTest)
numCandidates <- nrow(dfTest)

In [ ]:
# show a selection of data from the test...
head(dfRaw[c('systemname','A1L_7616_01#6789', 'A1L_20679_02#6790', 'A1L_5480_03#6791', 'S1', 'S2')])

## Summary Stats
We'll use the TAM package to get some summary info for the dataset. TAM will automatically run the PCM when our data is polytomous. We need to remove the systemname (candidate ID) column first.

In [ ]:
dfTestResponses <- dfRaw[-c(1)]

In [ ]:
pcm <- tam(dfTestResponses)

In [ ]:
summary(pcm)

## Item Difficulties
We can extract the item difficulties and use them to update the item bank if we wish.

In [ ]:
dfItemUpdates <- pcm$item

In [ ]:
head(dfItemUpdates[, c('item', 'N', 'M', 'xsi.item')])

Here, `N` is the number of responses, `M` is the ratio of correct responses and `xsi.item` is the item difficulty (or $b$ parameter).

In [ ]:
# read in items.csv and create a joined dataset using the new
# item difficulties calculated from the test response analysis
dfItems <- read_csv("data/items.csv") %>%
  select(-c(b, se)) %>%
  left_join(dfItemUpdates, by = c("UIID" = "item")) %>%
  select(c(UIID, a, b = xsi.item, rating, k))

# then write out to a new items file
write_csv(dfItems, 'data/updated-items.csv')

### Polytomous Item Thresholds
Under the GRM or PCM polytomous IRT models, items are given a difficulty however, a set of thresholds $K$ is also required in order to fully describe the item. There must be $K-1$ thresholds (with threshold $k_0$ not being required to be supplied).

In [ ]:
dfThresholds <- data.frame(tam.threshold(pcm))

In [ ]:
(dfThresholds)

In [ ]:
write_csv(dfThresholds, 'data/item-thresholds.csv')

## Item Fit
We can calulcate the infit and outfit statistics. For polytomous items, the statistics are produced for each level.

In [ ]:
Fit.poly <- tam.fit(pcm)

In [ ]:
head(Fit.poly$itemfit)

## Item Information Functions
If we're interested we can get Item Characteristic Curves (ICCs) and Item Information Functions (IIFs) for the polytomous item thresholds. Here we'll show the ICCs and IIFs for items S1 (item 113) and W1 (item 123).

In [ ]:
plot(pcm, items=113, type='expected', export=FALSE)

In [ ]:
plot(pcm, items=113, type='items', export=FALSE)

In [ ]:
plot(pcm, items=123, type='expected', export=FALSE)

In [ ]:
plot(pcm, items=123, type='items', export=FALSE)